In [1]:
import torch
from torch import nn, optim
from torch.nn import functional as F
import numpy as np

In [2]:
BATCH_SIZE = 4
EPOCHS = 10
INPUT_SIZE = 11
LAYER_SIZE = 10
LATENT_SIZE = 3

In [3]:
# Load the data from the file

saved = np.load("testing.npy", allow_pickle=True)

# Make a trainging and testing batch
train_data = torch.Tensor(saved[:int(len(saved)*0.1)])
test_data = torch.Tensor(saved[int(len(saved)*0.9):])
len(test_data)

10000

In [4]:
device = "cuda"

In [5]:
class VAE(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()
        
        # Encoder
        self.l1 = nn.Linear(INPUT_SIZE, LAYER_SIZE)
        self.l2a = nn.Linear(LAYER_SIZE, LATENT_SIZE)
        self.l2b = nn.Linear(LAYER_SIZE, LATENT_SIZE)
        
        # Decoder
        self.l3 = nn.Linear(LATENT_SIZE, LAYER_SIZE)
        self.l4 = nn.Linear(LAYER_SIZE, INPUT_SIZE)
        
        self.once = True
    
    # Run some data through the encoder
    def encode(self, x):
        out = F.relu(self.l1(x))
        # return the mu and the sigma
        return self.l2a(out), self.l2b(out)
    
    # The reparameterization trick, taken from https://github.com/pytorch/examples/blob/master/vae/main.py
    def reparameterize(self, mu, sigma):
        if self.once:
            print(mu, sigma)
            self.once - False
        std = torch.exp(0.5*sigma)
        eps = torch.randn_like(std)
        return mu + eps*std
    
    def decode(self, x):
        out = F.relu(self.l3(x))
        return torch.sigmoid(self.l4(out)) # sigmoid vs tanh
    
    def forward(self, x):
        # TODO: check the shape of x to be sure we have the right input
        mu, sigma = self.encode(x)
        z = self.reparameterize(mu, sigma)
        # The loss function needs the mu and the sigma so just return them here
        return self.decode(z), mu, sigma

In [6]:
# Reconstruction + KL divergence losses summed over all elements and batch
# Taken from https://github.com/pytorch/examples/blob/master/vae/main.py
def loss_function(recon_x, x, mu, sigma):
    BCE = F.binary_cross_entropy(recon_x, x, reduction='sum')
    KLD = -0.5 * torch.sum(1 + sigma - mu.pow(2) - sigma.exp())
    return BCE + KLD

In [7]:
why = VAE().to(device)
optimizer = optim.Adam(why.parameters(), lr=0.001)

In [8]:
def train(e_count):
    why.train()
    train_loss = 0
    for i in range(0, len(train_data), BATCH_SIZE):
        batch = train_data[i:i+BATCH_SIZE].to(device)
        why.zero_grad()
        recons, mu, sigma = why(batch)
        loss = loss_function(recons, batch, mu, sigma)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        
    print(f"Epoch: {e_count}, Loss: {train_loss/len(train_data)}")        
        
    
    

In [9]:
# Lets use CPU
def test(e_count):
    why.eval()
    test_loss = 0
    for i in range(0, len(test_data), BATCH_SIZE):
        batch = test_data[i:i+BATCH_SIZE].to(device)
        recons, mu, sigma = why(batch)
        loss = loss_function(recons, batch, mu, sigma)
        test_loss += loss.item()
    print(f"TEST Epoch: {e_count}, Loss: {test_loss/len(test_data)}") 
        
        

In [10]:
for i in range(EPOCHS):
    train(i)
for i in range(EPOCHS):
    test(i)

tensor([[-0.2112,  0.1469, -0.0175],
        [-0.2377,  0.1489, -0.0465],
        [-0.2383,  0.1288, -0.0584],
        [-0.1329,  0.1417, -0.0793]], device='cuda:0', grad_fn=<AddmmBackward>) tensor([[-0.0268,  0.0763, -0.1352],
        [-0.0052,  0.0722, -0.1383],
        [ 0.0135,  0.0691, -0.1479],
        [-0.0440,  0.1122, -0.2299]], device='cuda:0', grad_fn=<AddmmBackward>)
tensor([[-0.2384,  0.1457, -0.0493],
        [-0.2211,  0.1444, -0.0303],
        [-0.2155,  0.1485, -0.0306],
        [-0.2156,  0.1520, -0.0357]], device='cuda:0', grad_fn=<AddmmBackward>) tensor([[-0.0002,  0.0634, -0.1320],
        [-0.0142,  0.0659, -0.1299],
        [-0.0195,  0.0671, -0.1329],
        [-0.0202,  0.0674, -0.1358]], device='cuda:0', grad_fn=<AddmmBackward>)
tensor([[-0.2009,  0.1533, -0.0250],
        [-0.1245,  0.2215, -0.0527],
        [-0.1998,  0.1496, -0.0183],
        [-0.1953,  0.1574, -0.0248]], device='cuda:0', grad_fn=<AddmmBackward>) tensor([[-0.0280,  0.0705, -0.1360],
        

KeyboardInterrupt: 

In [ ]:
for i in range(EPOCHS):
    train(i)